In [101]:
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

In [102]:
dirPath = "F:/radiologyReportGeneration/datasets/orginal_iu_xray/ecgen-radiology"
fileList = os.listdir(dirPath)
filePathList = [dirPath + "/"+file for file in fileList]
filePathList[:5]

['F:/radiologyReportGeneration/datasets/orginal_iu_xray/ecgen-radiology/1.xml',
 'F:/radiologyReportGeneration/datasets/orginal_iu_xray/ecgen-radiology/10.xml',
 'F:/radiologyReportGeneration/datasets/orginal_iu_xray/ecgen-radiology/100.xml',
 'F:/radiologyReportGeneration/datasets/orginal_iu_xray/ecgen-radiology/1000.xml',
 'F:/radiologyReportGeneration/datasets/orginal_iu_xray/ecgen-radiology/1001.xml']

需要获取MeSH中的marge，automatic
imageid
report finding

In [103]:
# dataList = {"id": [], "merge":[], "automatic":[], "image_path":[], "report":[]}
dataList = []
for filePath in filePathList:
    mergeList = []
    imagesId = []
    report = ""
    impression = ""
    id = ""
    tree = ET.parse(filePath)
    root = tree.getroot()
    merge = root.find("MeSH").findall("major")
    automatic = root.find("MeSH").find("automatic")
    autoStr=None
    if automatic is not None:
        autoStr = automatic.text
    abstractTextEles = root.find("MedlineCitation").find("Article").find("Abstract").findall("AbstractText")
    parentImages = root.findall("parentImage")
    for m in merge:
        mergeList.append(m.text)
    for abstractText in abstractTextEles:
        if abstractText.get("Label") == "FINDINGS":
            report = abstractText.text
        if abstractText.get("Label") == "IMPRESSION":
            impression = abstractText.text
    for index, pi in enumerate(parentImages):
        if index ==0:
            id = pi.get("id").split("-")[0] + "-" + pi.get("id").split("-")[1]
        imagesId.append(id+f"/{index}.png")
    dataList.append([id, mergeList, autoStr, imagesId, impression, report])
dataDf = pd.DataFrame(dataList, columns=["id", "merge", "automatic", "image_path", "report", "impression"])
dataDf.head()

,id,merge,automatic,image_path,report,impression
0,CXR1_1_IM-0001,[normal],None,"[CXR1_1_IM-0001/0.png, CXR1_1_IM-0001/1.png]",Normal chest x-XXXX.,The cardiac silhouette and mediastinum size ar...
1,CXR10_IM-0002,[Calcified Granuloma/lung/upper lobe/right],calcified granuloma,"[CXR10_IM-0002/0.png, CXR10_IM-0002/1.png]",No acute cardiopulmonary process.,The cardiomediastinal silhouette is within nor...
2,CXR100_IM-0002,[normal],None,"[CXR100_IM-0002/0.png, CXR100_IM-0002/1.png]",No active disease.,Both lungs are clear and expanded. Heart and m...
3,CXR1000_IM-0003,"[Opacity/lung/upper lobe/right, Pulmonary Atel...",atelectases,"[CXR1000_IM-0003/0.png, CXR1000_IM-0003/1.png,...",1. Increased opacity in the right upper lobe w...,There is XXXX increased opacity within the rig...
4,CXR1001_IM-0004,[Markings/lung/bilateral/interstitial/diffuse/...,diffuse fibrosis,"[CXR1001_IM-0004/0.png, CXR1001_IM-0004/1.png]",Diffuse fibrosis. No visible focal acute disease.,Interstitial markings are diffusely prominent ...


In [104]:
dataDf[dataDf['id'] == "CXR1015_IM-0001"]

,id,merge,automatic,image_path,report,impression
19,CXR1015_IM-0001,"[Opacity/lung/base/bilateral/patchy/streaky, T...",atelectases,"[CXR1015_IM-0001/0.png, CXR1015_IM-0001/1.png,...","Bibasilar opacities, right greater than left, ...","Streaky and patchy bibasilar opacities, triang..."


In [105]:
mergeDf =  dataDf[["merge", "automatic"]].copy()
mergeDf["mergeLen"] = mergeDf["merge"].apply(lambda x: len(x))
maxMergeColumn = mergeDf["mergeLen"].max()

In [106]:
def getLabel(x, index):
    if index < len(x):
        return x[index]
    else:
        return None
for index in range(maxMergeColumn):
    mergeDf[f"merge{index}"] = mergeDf["merge"].apply(lambda x: getLabel(x, index))
mergeDf.head()

,merge,automatic,mergeLen,merge0,merge1,merge2,merge3,merge4,merge5,merge6,merge7,merge8,merge9,merge10,merge11,merge12
0,[normal],None,1,normal,None,None,None,None,None,None,None,None,None,None,None,None
1,[Calcified Granuloma/lung/upper lobe/right],calcified granuloma,1,Calcified Granuloma/lung/upper lobe/right,None,None,None,None,None,None,None,None,None,None,None,None
2,[normal],None,1,normal,None,None,None,None,None,None,None,None,None,None,None,None
3,"[Opacity/lung/upper lobe/right, Pulmonary Atel...",atelectases,3,Opacity/lung/upper lobe/right,Pulmonary Atelectasis/upper lobe/right,Opacity/lung/lingula,None,None,None,None,None,None,None,None,None,None
4,[Markings/lung/bilateral/interstitial/diffuse/...,diffuse fibrosis,2,Markings/lung/bilateral/interstitial/diffuse/p...,Fibrosis/diffuse,None,None,None,None,None,None,None,None,None,None,None


In [107]:
# mergeDf["merge0"].value_counts()
# mergeDf.to_excel("merge.xlsx", index=False)
labelDf = mergeDf["merge0"].apply(lambda x: x.split("/")[0])
labelDf.value_counts()

normal                 1391
Lung                    347
Cardiomegaly            261
Opacity                 195
Calcified Granuloma     175
                       ... 
Pneumoperitoneum          1
Contrast Media            1
Thoracic vertebrae        1
Hydropneumothorax         1
Lumbar Vertebrae          1
Name: merge0, Length: 92, dtype: int64

In [108]:
# labelDf.value_counts().to_csv("label.csv")

In [109]:
diseaseList = ['normal', 'Cardiomegaly','Pulmonary atelectasis','Calcified granuloma','Aorta/tortuous',
'Lung/hyperdistention','Opacity/lung','Pleural effusion','Lung/hypoinflation','Cicatrix/lung',
'Calcinosis/lung']
def getCLsLabel(x:list):
    result = []
    for x_i in x:
        for dcls in diseaseList:
            if dcls.lower().replace(" ", "") in x_i.lower().replace(" ", ""):
                result.append(dcls)
    return result
mergeDf["cls"] = mergeDf["merge"].apply(getCLsLabel)
lenDf = mergeDf["cls"].apply(lambda x: len(x)==0)
lenDf.value_counts()

False    2987
True      968
Name: cls, dtype: int64

In [110]:
mergeDf[lenDf]

,merge,automatic,mergeLen,merge0,merge1,merge2,merge3,merge4,merge5,merge6,merge7,merge8,merge9,merge10,merge11,merge12,cls
4,[Markings/lung/bilateral/interstitial/diffuse/...,diffuse fibrosis,2,Markings/lung/bilateral/interstitial/diffuse/p...,Fibrosis/diffuse,None,None,None,None,None,None,None,None,None,None,None,[]
5,[Mastectomy/left],mastectomies,1,Mastectomy/left,None,None,None,None,None,None,None,None,None,None,None,None,[]
10,[Thoracic Vertebrae/degenerative/mild],degenerative change,1,Thoracic Vertebrae/degenerative/mild,None,None,None,None,None,None,None,None,None,None,None,None,[]
23,[Costophrenic Angle/right/blunted],effusion,1,Costophrenic Angle/right/blunted,None,None,None,None,None,None,None,None,None,None,None,None,[]
28,[No Indexing],None,1,No Indexing,None,None,None,None,None,None,None,None,None,None,None,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944,[Foreign Bodies/thorax/left],foreign body,1,Foreign Bodies/thorax/left,None,None,None,None,None,None,None,None,None,None,None,None,[]
3947,"[Nodule/lung/bilateral/multiple, Surgical Inst...",metastatic disease,3,Nodule/lung/bilateral/multiple,Surgical Instruments/left,"Catheters, Indwelling/right",None,None,None,None,None,None,None,None,None,None,[]
3950,[Sutures/sternum],bypass grafts,1,Sutures/sternum,None,None,None,None,None,None,None,None,None,None,None,None,[]
3952,"[Calcinosis/mediastinum, Thoracic Vertebrae/de...",degenerative change,2,Calcinosis/mediastinum,Thoracic Vertebrae/degenerative,None,None,None,None,None,None,None,None,None,None,None,[]


In [111]:
# mergeDf[lenDf].to_csv("label_fail.csv", index=False)

In [112]:
mergeDf["cls"].value_counts()

[normal]                                                              1391
[]                                                                     968
[Calcified granuloma]                                                  167
[Cardiomegaly]                                                         165
[Lung/hypoinflation]                                                   126
                                                                      ... 
[Opacity/lung, Pulmonary atelectasis, Cicatrix/lung, Cardiomegaly]       1
[Calcinosis/lung, Cicatrix/lung]                                         1
[Cicatrix/lung, Cardiomegaly]                                            1
[Pleural effusion, Cardiomegaly, Pulmonary atelectasis]                  1
[Pulmonary atelectasis, Pulmonary atelectasis]                           1
Name: cls, Length: 260, dtype: int64

In [113]:
for disea in diseaseList:
    print(disea)
    tempDf = mergeDf["cls"].apply(lambda x: disea in x)
    print(len(tempDf[tempDf]))

normal
1396
Cardiomegaly
375
Pulmonary atelectasis
332
Calcified granuloma
274
Aorta/tortuous
159
Lung/hyperdistention
208
Opacity/lung
406
Pleural effusion
161
Lung/hypoinflation
283
Cicatrix/lung
168
Calcinosis/lung
132


In [114]:
# 把对应的报告筛选出来，构建关系
mergeDf.loc[lenDf, "cls"] = "other"

### 构建实体关系

In [115]:
def mergeSplit(x):
    result = []
    merge = x["merge"]
    automatic = x["automatic"]
    for m in merge:
        result += m.split("/")
    if automatic:
        result += automatic
    return result

In [116]:
dataDf["impression"] = dataDf["impression"].fillna("")
dataDf["report"] = dataDf["report"].fillna("")

In [117]:
dataDf["cls"] = mergeDf["cls"]
finding_result = {}
for disea in diseaseList:
    diseaIndex = dataDf["cls"].apply(lambda x: disea in x)
    diseaDf = dataDf[diseaIndex]
    keywords = diseaDf[["automatic", "merge"]].apply(mergeSplit, axis=1)
    finding_result[disea] = {"info": (diseaDf["report"] +" "+ diseaDf["impression"]).tolist(),
                            "keyWord": keywords.tolist()}
diseaDf = dataDf[dataDf["cls"]=="other"]
keywords = diseaDf[["automatic", "merge"]].apply(mergeSplit, axis=1)
finding_result["other"] = {"info": (diseaDf["report"] +" "+ diseaDf["impression"]).tolist(),"keyWord": keywords.tolist()}

In [118]:
dataDf.head()

,id,merge,automatic,image_path,report,impression,cls
0,CXR1_1_IM-0001,[normal],None,"[CXR1_1_IM-0001/0.png, CXR1_1_IM-0001/1.png]",Normal chest x-XXXX.,The cardiac silhouette and mediastinum size ar...,[normal]
1,CXR10_IM-0002,[Calcified Granuloma/lung/upper lobe/right],calcified granuloma,"[CXR10_IM-0002/0.png, CXR10_IM-0002/1.png]",No acute cardiopulmonary process.,The cardiomediastinal silhouette is within nor...,[Calcified granuloma]
2,CXR100_IM-0002,[normal],None,"[CXR100_IM-0002/0.png, CXR100_IM-0002/1.png]",No active disease.,Both lungs are clear and expanded. Heart and m...,[normal]
3,CXR1000_IM-0003,"[Opacity/lung/upper lobe/right, Pulmonary Atel...",atelectases,"[CXR1000_IM-0003/0.png, CXR1000_IM-0003/1.png,...",1. Increased opacity in the right upper lobe w...,There is XXXX increased opacity within the rig...,"[Opacity/lung, Pulmonary atelectasis, Opacity/..."
4,CXR1001_IM-0004,[Markings/lung/bilateral/interstitial/diffuse/...,diffuse fibrosis,"[CXR1001_IM-0004/0.png, CXR1001_IM-0004/1.png]",Diffuse fibrosis. No visible focal acute disease.,Interstitial markings are diffusely prominent ...,other


dataDf.to_csv("output/iu_data.csv", index=False)

In [119]:
# 数据准备
# file_path = "iu_data.json"
# with open(file_path, 'r') as f:
#     iu_data = json.load(f)
# for key in iu_data.keys():
#     infos = iu_data.get(key).get("info")
#     disea = key.replace("/", "_")
#     with open(f"KLGraph/{disea}_report.txt", 'w') as f:
#         for info in infos:
#             f.write(info + "\n")
#         f.close()
def extract_cls(x):
    if isinstance(x, list):
        return x[0]
    else:
        return x

dataDf["n_cls"] = dataDf["cls"].apply(extract_cls)
dataDf.head()

,id,merge,automatic,image_path,report,impression,cls,n_cls
0,CXR1_1_IM-0001,[normal],None,"[CXR1_1_IM-0001/0.png, CXR1_1_IM-0001/1.png]",Normal chest x-XXXX.,The cardiac silhouette and mediastinum size ar...,[normal],normal
1,CXR10_IM-0002,[Calcified Granuloma/lung/upper lobe/right],calcified granuloma,"[CXR10_IM-0002/0.png, CXR10_IM-0002/1.png]",No acute cardiopulmonary process.,The cardiomediastinal silhouette is within nor...,[Calcified granuloma],Calcified granuloma
2,CXR100_IM-0002,[normal],None,"[CXR100_IM-0002/0.png, CXR100_IM-0002/1.png]",No active disease.,Both lungs are clear and expanded. Heart and m...,[normal],normal
3,CXR1000_IM-0003,"[Opacity/lung/upper lobe/right, Pulmonary Atel...",atelectases,"[CXR1000_IM-0003/0.png, CXR1000_IM-0003/1.png,...",1. Increased opacity in the right upper lobe w...,There is XXXX increased opacity within the rig...,"[Opacity/lung, Pulmonary atelectasis, Opacity/...",Opacity/lung
4,CXR1001_IM-0004,[Markings/lung/bilateral/interstitial/diffuse/...,diffuse fibrosis,"[CXR1001_IM-0004/0.png, CXR1001_IM-0004/1.png]",Diffuse fibrosis. No visible focal acute disease.,Interstitial markings are diffusely prominent ...,other,other


In [120]:
dataDf["n_cls"].value_counts()

normal                   1392
other                     968
Cardiomegaly              325
Opacity/lung              244
Lung/hypoinflation        222
Calcified granuloma       213
Lung/hyperdistention      182
Aorta/tortuous            110
Pulmonary atelectasis      85
Calcinosis/lung            80
Pleural effusion           71
Cicatrix/lung              63
Name: n_cls, dtype: int64

# 读取annotation.json 加上cls信息

In [121]:
import json
with open("F:/radiologyReportGeneration/datasets/iu_xray/annotation.json", "r") as f:
    data = json.load(f)
datasetSplit = list(data.keys())

In [122]:
dataDf[dataDf["id"]==""]

,id,merge,automatic,image_path,report,impression,cls,n_cls
77,,[normal],None,[],No active disease.,Heart size and pulmonary vascularity are withi...,[normal],normal
91,,"[Implanted Medical Device/left, Cardiomegaly, ...",atelectases,[],1. Stable cardiomegaly with left basilar infil...,There is a pacemaker left chest wall with stab...,"[Cardiomegaly, Opacity/lung, Pulmonary atelect...",Cardiomegaly
142,,[Lung/hypoinflation],None,[],1. No acute cardiopulmonary disease.,The heart and mediastinum are unremarkable. Th...,[Lung/hypoinflation],Lung/hypoinflation
201,,"[Implanted Medical Device, Cardiomegaly, Pleur...",opacity,[],1. Cardiomegaly without overt heart failure. 2...,The central venous line has been removed. An i...,"[Cardiomegaly, Pleural effusion, Opacity/lung]",Cardiomegaly
205,,"[Medical Device/left, Opacity/lung/base/right,...",atelectases,[],Vagal nerve stimulator overlying the left ches...,There is a vagal nerve stimulator with XXXX ov...,"[Opacity/lung, Pulmonary atelectasis]",Opacity/lung
...,...,...,...,...,...,...,...,...
3815,,[normal],None,[],No acute cardiopulmonary abnormality..,"The lungs are clear bilaterally. Specifically,...",[normal],normal
3818,,[Density/lung/lower lobe],None,[],No acute cardiopulmonary abnormality identified.,2 images. Vague density within the lower lung ...,other,other
3839,,"[Pneumothorax/right, Mediastinal Emphysema, Fr...",comminuted fracture,[],The XXXX right pneumothorax and XXXX pneumomed...,Frontal and lateral views of the chest show no...,other,other
3842,,[No Indexing],None,[],No active disease.,The heart and lungs have XXXX XXXX in the inte...,other,other


In [123]:
dataDf = dataDf[dataDf["id"]!= ""]
dataDf["n_cls"].value_counts()

normal                   1380
other                     944
Cardiomegaly              304
Opacity/lung              233
Lung/hypoinflation        217
Calcified granuloma       209
Lung/hyperdistention      178
Aorta/tortuous            103
Pulmonary atelectasis      80
Calcinosis/lung            77
Pleural effusion           66
Cicatrix/lung              60
Name: n_cls, dtype: int64

In [129]:
clsMap = {}
with open("../data/cls.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.replace("\n", "").split("\t")
        clsMap[line[1]] = line[0]
clsList = []
for dataType in datasetSplit:
    dataS = data.get(dataType)
    for info in dataS:
        id = info.get("id")
        if len(id.split("-")) > 2:
            id = id.split("-")[0] + "-" + id.split("-")[1]
        if  len(dataDf.loc[dataDf["id"]==id])!=1:
            print(id)
            continue
        cls = dataDf.loc[dataDf["id"].str.contains(id), 'n_cls'].item()
        cls = cls.replace(" ", "_").replace("/", "_")
        if clsMap.get(cls) is None:
            print(cls)
        clsId = int(clsMap.get(cls))
        clsList.append(clsId)
        info["cls"] = clsId

In [131]:
with open("../data/annotation.json", 'w') as file:
    json.dump(data, file)